<a href="https://colab.research.google.com/github/mvadrev/coviScan/blob/main/Extracting_spectrogram_coswara.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Coswara Spectrogram CNN Dataset Preparation

In [ ]:
!git clone https://github.com/iiscleap/Coswara-Data.git
%cd Coswara-Data

Cloning into 'Coswara-Data'...
remote: Enumerating objects: 762, done.
remote: Total 762 (delta 0), reused 0 (delta 0), pack-reused 762
Receiving objects: 100% (762/762), 13.99 GiB | 23.53 MiB/s, done.
Resolving deltas: 100% (302/302), done.
Checking out files: 100% (178/178), done.
/content/Coswara-Data


Let's extract them

In [ ]:
!python extract_data.py

Streaming output truncated to the last 5000 lines.
20200707/6WVdsp5LDLRRfYrfiR4BU2G6wve2/cough-heavy.wav
20200707/6WVdsp5LDLRRfYrfiR4BU2G6wve2/cough-shallow.wav
20200707/sW80fLKGSiXcPBKFwnwRaaRdPKX2/counting-normal.wav
20200707/sW80fLKGSiXcPBKFwnwRaaRdPKX2/metadata.json
20200707/sW80fLKGSiXcPBKFwnwRaaRdPKX2/vowel-o.wav
20200707/sW80fLKGSiXcPBKFwnwRaaRdPKX2/vowel-a.wav
20200707/sW80fLKGSiXcPBKFwnwRaaRdPKX2/vowel-e.wav
20200707/sW80fLKGSiXcPBKFwnwRaaRdPKX2/breathing-shallow.wav
20200707/sW80fLKGSiXcPBKFwnwRaaRdPKX2/counting-fast.wav
20200707/sW80fLKGSiXcPBKFwnwRaaRdPKX2/breathing-deep.wav
20200707/sW80fLKGSiXcPBKFwnwRaaRdPKX2/cough-heavy.wav
20200707/sW80fLKGSiXcPBKFwnwRaaRdPKX2/cough-shallow.wav
20200707/f4wYeTyCImQ0BQiwBOlScS9C2H03/counting-normal.wav
20200707/f4wYeTyCImQ0BQiwBOlScS9C2H03/metadata.json
20200707/f4wYeTyCImQ0BQiwBOlScS9C2H03/vowel-o.wav
20200707/f4wYeTyCImQ0BQiwBOlScS9C2H03/vowel-a.wav
20200707/f4wYeTyCImQ0BQiwBOlScS9C2H03/vowel-e.wav
20200707/f4wYeTyCImQ0BQiwBOlScS9C2H0

In [ ]:
import os
from pathlib import Path
import shutil

import librosa
import librosa.display
import numpy as np
import tqdm
from matplotlib import pyplot as plt
from PIL import Image

# https://stackoverflow.com/questions/56719138/how-can-i-save-a-librosa-spectrogram-plot-as-a-specific-sized-image/57204349
def scale_minmax(X, min=0.0, max=1.0):
    X_std = (X - X.min()) / (X.max() - X.min())
    X_scaled = X_std * (max - min) + min
    return X_scaled


def extract_spectrogram_image(file_path, out_path, sr=22050, time_steps=384,
                              duration=5, hop_length=512, n_mels=128):
    y, sr = librosa.load(file_path, duration=duration, sr=sr)

    # extract a fixed length window
    start_sample = 0  # starting at beginning
    length_samples = time_steps*hop_length
    window = y[start_sample:start_sample+length_samples]

    # use log-melspectrogram
    mels = librosa.feature.melspectrogram(y=window, sr=sr, n_mels=n_mels,
                                            n_fft=hop_length*2, hop_length=hop_length)
    mels = np.log(mels + 1e-9)  # add small number to avoid log(0)

    # min-max scale to fit inside 8-bit range
    img = scale_minmax(mels, 0, 255).astype(np.uint8)
    img = np.flip(img, axis=0) # put low frequencies at the bottom in image
    img = 255 - img # invert. make black==more energy

    # save as PNG
    # skimage.io.imsave(out, img)
    Image.fromarray(img).save(out_path)

out_dir = 'Coswara-Processed'
out_image_dir = os.path.join(out_dir, 'images')
os.makedirs(out_image_dir, exist_ok=True)
!rm -rf {out_dir}

data_dir = Path('Extracted_data')
audio_files = list(data_dir.glob('*/*/cough-*.wav'))
label_file = 'combined_data.csv'

mono = True
duration = 5
sr = 22050
n_mels = 128  # The height
time_steps = 384  # The width
hop_length = 512
error_cases = []
for file_path in tqdm.tqdm(audio_files):
    file_path = str(file_path)

    # Build output file path
    filename = os.path.splitext(os.path.basename(file_path))[0]
    user_id = file_path.split(os.sep)[-2]
    out_user_dir = os.path.join(out_image_dir, user_id)
    if not os.path.exists(out_user_dir):
        os.makedirs(out_user_dir, exist_ok=True)
    out_path = os.path.join(out_user_dir, filename + '.png')

    if os.path.exists(out_path):
        continue

    try:
        extract_spectrogram_image(file_path, out_path, sr, time_steps, duration, hop_length, n_mels)
    except Exception as e:
        print(f'Failed while processing {file_path} with exception: {e}')
        error_cases.append(user_id)
        shutil.rmtree(out_user_dir)

KeyboardInterrupt: ignored

In [ ]:
print('Total errors:', len(error_cases))

Total errors: 65


In [ ]:
import pandas as pd
out_label_file = "mukund.csv"
df = pd.read_csv(label_file)
print('Original data:', df.shape)
df = df[~df['id'].isin(error_cases)]
print('Clean data:', df.shape)
df.to_csv(out_label_file)

Original data: (2233, 36)
Clean data: (2192, 36)


## Compress and move it to Google Drive

In [ ]:
!tar -czvf Coswara-Processed.tar.gz Coswara-Processed

Streaming output truncated to the last 5000 lines.
Coswara-Processed/images/Axei3vO2YcQM56PrwqmMF40A6aG3/
Coswara-Processed/images/Axei3vO2YcQM56PrwqmMF40A6aG3/cough-heavy.png
Coswara-Processed/images/Axei3vO2YcQM56PrwqmMF40A6aG3/cough-shallow.png
Coswara-Processed/images/HmsAhPiHhiQOnvXYXzZ5u5qFqLq1/
Coswara-Processed/images/HmsAhPiHhiQOnvXYXzZ5u5qFqLq1/cough-heavy.png
Coswara-Processed/images/HmsAhPiHhiQOnvXYXzZ5u5qFqLq1/cough-shallow.png
Coswara-Processed/images/3LCqXTY14jROmsMpG3XkEiMJ9cD3/
Coswara-Processed/images/3LCqXTY14jROmsMpG3XkEiMJ9cD3/cough-heavy.png
Coswara-Processed/images/3LCqXTY14jROmsMpG3XkEiMJ9cD3/cough-shallow.png
Coswara-Processed/images/4w3GC0eRNVXJ4q4vRLphW9RIiQY2/
Coswara-Processed/images/4w3GC0eRNVXJ4q4vRLphW9RIiQY2/cough-heavy.png
Coswara-Processed/images/4w3GC0eRNVXJ4q4vRLphW9RIiQY2/cough-shallow.png
Coswara-Processed/images/tDjXrO1dI4O35JJXPL3ihLvnAzl1/
Coswara-Processed/images/tDjXrO1dI4O35JJXPL3ihLvnAzl1/cough-heavy.png
Coswara-Processed/images/tDjXrO1dI4O

In [ ]:
!rm -rf /content/Coswara-Data/Coswara-Processed.tar.gz

In [ ]:
from google.colab import drive

drive.mount('/content/drive/')

KeyboardInterrupt: ignored

In [ ]:
!cp Coswara-Processed.tar.gz /content/drive/MyDrive

cp: cannot create regular file '/content/drive/MyDrive': No such file or directory


In [ ]:
!du -h Coswara-Processed.tar.gz

62M	Coswara-Processed.tar.gz
